In [15]:
from dotenv import load_dotenv      # loads environment variables from a .env file into Python environment
import os                           # access environment variables once they've been loaded into the Python environmen
from langchain_groq import ChatGroq   # Access Groq chat models

In [16]:
# print(os.getenv('GROQ_API_KEY'))

In [17]:
from langchain_core.prompts import ChatPromptTemplate             

# Creating a template for the prompt
template = """
You are a MySQL data analyst. Generate an SQL query to answer the given question based on the provided schema.

Instructions:
- Return only the SQL query, without explanations or additional formatting.
- Ensure the query is on a single line with no newline characters (`\n`) or backslashes (`\`).

Example:
Question: How many customers are there?
Output: SELECT COUNT(*) FROM customers;

Now, generate the SQL query:

Schema: {schema}    

Question: {question}

SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)


In [18]:
# prompt.format(schema= "myshema", question="my question")

In [19]:
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

# Define the connection URI
mysql_uri = "mysql+mysqlconnector://root:@localhost:3306/classicmodels"

# Create an SQLAlchemy engine
engine = create_engine(mysql_uri)

# Pass the engine to SQLDatabase
db = SQLDatabase(engine)  # db is an instance of SQLDatabase


In [20]:
# db.run("SQL Query") # runs the SQL query and returns the result
db.run("show tables") # returns a list of tables in the database

"[('customers',), ('employees',), ('offices',), ('orderdetails',), ('orders',), ('payments',), ('productlines',), ('products',)]"

In [21]:
# Get the schema of the database
# Schema is a list of dictionaries, where each dictionary represents a column in a table

def get_schema(_):    # For the RunnablePassthrough towork function should be consist with at least one parameter
    schema = db.get_table_info()
    return schema

In [22]:
# get_schema(None)

In [23]:
from langchain_core.output_parsers import StrOutputParser  # Get the Out Put as a String
from langchain_core.runnables import RunnablePassthrough   
from langchain_openai import ChatOpenAI                    # Importing the OpenAI Chat Model

# llm = ChatOpenAI(
#     temperature=0.5,
#     max_tokens=100,
#     model = "gpt-3.5-turbo"

# )

llm = ChatGroq(model="llama3-8b-8192",temperature = 0.5)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema) 
    | prompt
    | llm
    | StrOutputParser()
)


In [24]:
user_question = 'How many customer are there'
sql_chain.invoke({"question": user_question})

'SELECT COUNT(*) FROM customers;'

In [25]:
template = """Think you are Data Analyst and use Mysql databases. Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [26]:
def run_query(query):
    return db.run(query)


In [27]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema = get_schema,
        response = lambda vars: run_query(vars["query"])
    )
    | prompt_response
    | llm
    | StrOutputParser()
)


In [28]:
user_question = 'Give me the customer name, phone and payment amount greater than 100000?'
full_chain.invoke({"question": user_question})

'Here is the natural language response:\n\nBased on the provided database, we can identify customers who have made payments greater than $100,000. After running the SQL query, we found that the following customers have made such payments:\n\n* Mini Gifts Distributors Ltd. with a phone number of 4155551450 and payment amounts of $101,244.59 and $111,654.40.\n* Euro+ Shopping Channel with a phone number of (91) 555 94 44 and payment amounts of $116,208.40 and $120,166.58.\n* Dragon Souveniers, Ltd. with a phone number of +65 221 7555 and a payment amount of $105,743.00.\n\nThese customers have made significant payments, which may indicate large transactions or long-term relationships with the company.'